In [1]:
# Imports
!pip install evaluate
import evaluate
import math
import pandas as pd
import os
import re
import time
import torch, torch._dynamo
torch.set_float32_matmul_precision('high')
torch._dynamo.config.cache_size_limit = 128     # ← o 0 / False según quieras
#torch._dynamo.config.enabled = False         # ← si prefieres desactivar
import tensorflow as tf
from enum import Enum
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from typing import Literal

# Para no ver el warning de temperature
from transformers import logging
logging.set_verbosity_error()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00


In [2]:
print(tf.__version__)
tf.config.list_physical_devices('GPU')
!nvidia-smi

2.18.0
Wed Jul 16 17:47:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             49W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+----------------------------------------

In [3]:
!pip show transformers
!pip show torch

Name: transformers
Version: 4.53.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-c

In [ ]:
HUGGINGFACE_TOKEN=os.environ.get("HUGGINGFACE_TOKEN")
# Enum de los modelos
class ModelsEnum(str, Enum):
    BERT_BASE = "bert-base-uncased"
    MODERN_BERT = "answerdotai/ModernBERT-base"


class ModelsGenerativeEnum(str, Enum):
    GEMMA = "google/gemma-2-2b"
    LLAMA = "meta-llama/Llama-3.2-1B"
    LLAMA_SEXISM = "MatteoWood/llama-sexism-classifier-v1"
    MISTRAL = "mistralai/Mistral-7B-Instruct-v0.3"


In [5]:
def get_sexism_binary_classification_prompt(text: str) -> str:
    """
    Get the prompt for the sexism binary classification task.
    """
    return f"""
[System]
You are a STRICT text-classifier that detects sexism. Return **one digit** only: '0' or '1'. Nothing else.

- Output '1' (sexist): Any explicit insult, stereotype, hostility or declaration that women are inferior, worthless or only sexual objects.
- Output '0' (not sexist): Neutral, positive or negative content **not** aimed at women for being women.

### Examples
Text: 'Spread awareness that being a good person wont get you anywhere with femoids.'
Answer: 1
Reason: Uses the slur “femoids” to de-humanise women and asserts hostility toward them.
---
Text: 'Are you a female?'
Label: 0
Reason: A neutral question about the person’s gender; no negative judgement.
---
Text: 'Notice how women never shut the fuck up?'
Answer: 1
Reason: Hostile generalisation that all women talk too much; explicit insult.
---
Text: 'The judge should be the victim of the female serial killer.'
Answer: 0
Reason: The threat targets the judge, not women; “female” is only descriptive.
---
Text: 'Go fuck yourselves. I'm so sick of women like this piece of shit.'
Answer: 1
Reason: Direct profanity and hatred aimed specifically at women.
---
Text: 'Horror as christian woman stabbed for reading the bible in migrant centre.'
Answer: 0
Reason: A factual headline; mentions a woman but contains no sexist content.
### End examples

### Text to classify
{text}
### Output
(one digit: 0 or 1)
""".strip()

In [6]:
def sexism_3label_prompt(text: str) -> str:
    return f"""
[System]
You are a STRICT text-classifier that detects sexism. Return **one digit** only: '0', '1', or '2'. Nothing else.

- Output '2' (sexist): Any explicit insult, stereotype, hostility or declaration that women are inferior, worthless or only sexual objects.
- Output '1' (unsure): Borderline / implicit sexism, sexualisation, mild slur or you genuinely can’t decide if sexism is present.
- Output '0' (not sexist): Neutral, positive or negative content **not** aimed at women for being women.

### Examples
Text: 'Spread awareness that being a good person wont get you anywhere with femoids.'
Answer: 2
Reason: Uses the slur “femoids” to de-humanise women and asserts hostility toward them.
---
Text: 'Habits are hard to die, as much as the cock carousel she rode all her life.'
Label: 1
Reason: It's implying she is a slut.
---
Text: 'Are you a female?'
Label: 0
Reason: A neutral question about the person’s gender; no negative judgement.
---
Text: 'Notice how women never shut the fuck up?'
Answer: 2
Reason: Hostile generalisation that all women talk too much; explicit insult.
---
Text: 'Most here would fuck her.'
Answer: 1
Reason: Backhanded gendered compliments.
---
Text: 'The judge should be the victim of the female serial killer.'
Answer: 0
Reason: The threat targets the judge, not women; “female” is only descriptive.
---
Text: 'Go fuck yourselves. I'm so sick of women like this piece of shit.'
Answer: 2
Reason: Direct profanity and hatred aimed specifically at women.
---
Text: 'She loves but you, but since she's young she still wants to fuck other guys. Drop her.'
Answer: 1
Reason: Implies that young women are inherently promiscuous — a negative stereotype.
---
Text: 'Horror as christian woman stabbed for reading the bible in migrant centre.'
Answer: 0
Reason: A factual headline; mentions a woman but contains no sexist content.
### End examples

### Text to classify
{text}
### Output
(one digit: 0, 1 or 2)
""".strip()


In [7]:
def sexism_4label_prompt(text: str) -> str:
    return f"""
[System]
You are a STRICT text-classifier that detects sexism and its annotation consensus level.
Return **one digit only**: '0', '1', '2' or '3'. Nothing else.

- Output '3' (sexist – high consensus): All three annotators agreed the text is sexist.
- Output '2' (sexist – medium consensus): Two out of three annotators judged it sexist.
- Output '1' (sexist – low consensus): Only one annotator judged it sexist (borderline / unclear).
- Output '0' (not sexist): All three annotators agreed it is **not** sexist.

Guidelines (apply to decide between 0 vs 1-3):
- *Sexist* includes insults, stereotypes, hostility or objectification directed at women **for being women**.
- *Not sexist* may mention women neutrally/positively or criticise an individual without implying gender inferiority.
- If signs of sexism are present but ambiguous = 1.
- The stronger / more explicit the sexism, the higher the digit (2 or 3).

### Examples
Text: 'Spread awareness that being a good person wont get you anywhere with femoids.'
Answer: 3
Reason: Uses the slur “femoids” to de-humanise women and asserts hostility toward them.
---
Text: 'Habits are hard to die, as much as the cock carousel she rode all her life.'
Label: 2
Reason: It's implying she is a slut.
---
Text: 'One small way of replying to this manipulation get sterilized. Do not breed with these women, ever.'
Label: 1
Reason: Calls women manipulative 'breeders' and urges men to avoid them.
---
Text: 'Are you a female?'
Label: 0
Reason: A neutral question about the person’s gender; no negative judgement.
---
Text: 'Notice how women never shut the fuck up?'
Answer: 3
Reason: Hostile generalisation that all women talk too much; explicit insult.
---
Text: 'Most here would fuck her.'
Answer: 2
Reason: Backhanded gendered compliments.
---
Text: 'I am actually glad they made women clean that.'
Label: 1
Reason: Applauds the stereotype that cleaning is women’s job.
---
Text: 'The judge should be the victim of the female serial killer.'
Answer: 0
Reason: The threat targets the judge, not women; “female” is only descriptive.
---
Text: 'Go fuck yourselves. I'm so sick of women like this piece of shit.'
Answer: 3
Reason: Direct profanity and hatred aimed specifically at women.
---
Text: 'Won't miss her liberal wrinkly ass! Bye bitch.'
Label: 2
Reason: Uses the gendered slur 'bitch' alongside demeaning body comments.
---
Text: 'She loves but you, but since she's young she still wants to fuck other guys. Drop her.'
Answer: 1
Reason: Implies that young women are inherently promiscuous — a negative stereotype.
---
Text: 'Horror as christian woman stabbed for reading the bible in migrant centre.'
Answer: 0
Reason: A factual headline; mentions a woman but contains no sexist content.
### End examples

### Text to classify
{text}
### Output
(one digit: 0, 1, 2 or 3)
""".strip()


In [ ]:
from transformers import LogitsProcessor

class DigitsOnly(LogitsProcessor):
    def __init__(self, tokenizer):
        super().__init__()
        self.allowed_ids = [
            tokenizer.convert_tokens_to_ids(d) for d in ["0", "1", "2", "3"]
        ]
    def __call__(self, input_ids, scores):
        mask = scores.new_full(scores.shape, float("-inf"))
        mask[..., self.allowed_ids] = 0
        return scores + mask


def parse_llm_output(generated: str) -> int:
    """
    Devuelve 0-3 si el modelo lo responde; 4 si la salida está vacía
    o no contiene un dígito válido.
    """
    if not generated or not generated.strip():
        print("Cadena vacía")
        print(generated.strip())
        return 4
    first = generated.strip().splitlines()[0].strip()
    if first in {"0", "1", "2", "3"}:
      return int(first)
    else:
      print("No hay match")
      print(first)
      return 4



def predict_sexism_generative_model(
    model_name: ModelsGenerativeEnum,
    texts: list[str],
    classification_prompt: Literal["binary", "3labels", "4labels"] = "binary",
    batch_size: int = 8,
    max_new_tokens: int = 4,
):
    tokenizer = AutoTokenizer.from_pretrained(model_name.value, token=HUGGINGFACE_TOKEN)
    logits_proc = [DigitsOnly(tokenizer)]
    model = AutoModelForCausalLM.from_pretrained(
        model_name.value,
        token=HUGGINGFACE_TOKEN,
        torch_dtype=torch.bfloat16,      # opcional: más ligero
        device_map="auto",               # deja que HF coloque en GPU
    )

    # ---------- 1) return_full_text=False & GPU index correcto ----------
    device_idx = 0 if torch.cuda.is_available() else -1
    import copy

    gen_cfg = copy.deepcopy(model.generation_config)   # <- en vez de .clone()
    gen_cfg.temperature = None
    gen_cfg.top_p = None
    gen_cfg.top_k = None
    gen_cfg.do_sample = False

    model.generation_config = gen_cfg

    text_gen = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
    )

    predictions = []
    num_batches = math.ceil(len(texts) / batch_size)
    for b in range(num_batches):
        print(f"Batch {b + 1}/{num_batches}")
        # Medimos el tiempo de ejecución
        start_time = time.time()
        batch_texts = texts[b * batch_size : (b + 1) * batch_size]

        # Para cada clasificación ponemos un prompt distinto
        if classification_prompt == "binary":
            prompts = [get_sexism_binary_classification_prompt(t) for t in batch_texts]
        elif classification_prompt == "3labels":
            prompts = [sexism_3label_prompt(t) for t in batch_texts]
        elif classification_prompt == "4labels":
            prompts = [sexism_4label_prompt(t) for t in batch_texts]
        else:
            raise ValueError("Prompt type not recognised")

        # Generamos el output
        outputs = text_gen(
            prompts,
            max_new_tokens=1,
            do_sample=False,
            logits_processor=logits_proc
        )

        # Parse del resultado
        predictions.extend(parse_llm_output(o[0]["generated_text"]) for o in outputs)
        print(f"\tTime: {time.time() - start_time:.2f} seconds")

    return predictions

In [9]:
# Como estoy haciéndolo desde drive y cargo los datos desde ahí
from google.colab import drive
drive.mount('/content/drive')
carpeta = '/content/drive/My Drive/Facultad/Máster/TFM/Datos'

Mounted at /content/drive


In [10]:
from typing import Literal
def generative_classifier(
    model: ModelsGenerativeEnum,
    dataset: str,
    classification_prompt: Literal["binary", "3labels", "4labels"] = "binary",
    batch_size: int = 8,
    limit: int = 200,
):
    """
    Make predictions with a Generative AI model for sexism detection.
    """
    try:
        # Cargar el dataset
        df = pd.read_csv(f"{carpeta}/{dataset}")

        # Ordenamos por longitud del texto
        df = df.sort_values("text", key=lambda s: s.str.len())

        # Se queda solo con los de test si es EDOS
        if "edos" in dataset:
          df = df[df["split"] == "test"]
        print(df.head())

        # Archivo de predicciones
        base_name   = model.value.split("/")[-1]
        predictions_folder = f"{carpeta}/{dataset.replace('.csv', '')}"
        if not os.path.exists(predictions_folder):
          os.makedirs(predictions_folder)
        predictions_file = f"{carpeta}/{dataset.replace('.csv', '')}/predictions_{classification_prompt}_{base_name}.csv"

        # Obtenemos la lista de textos
        texts = df["text"].tolist()

        # Calculamos las predicciones
        df["preds"] = predict_sexism_generative_model(
            model_name=model,
            texts=texts,
            classification_prompt=classification_prompt,
            batch_size=batch_size,
        )

        # Guardamos el archivo CSV con los resultados
        df.to_csv(predictions_file, index=False)
        print("Predicciones guardadas en", predictions_file)

        return {"message": f"Terminado para {model.name}. Nuevas filas calculadas: {len(texts)}"}

    except Exception as e:
        print("ERROR:", e)
        raise

In [11]:
texts = [
"I contacted her but never heard from her so I chalked it up to nothing",
"I always wanted to see one of those so I could run up, pull out my cock and fuck her face.",
"If a female is having sex and relationships they do not have social anxiety.",
"You incels bitch about how terrible and miserable and psychologically-damaging your lives are because you don't have sex, but as soon as a woman has surgery to get *out* of that situation, she's selfish?",
"That hug could turn into a false sexual assault or rape conviction down the road....this is damn good advice for men...never touch any girl or woman....let them wither on the vine from lack of attention.",
"It's fucked up how women aren't allowed to even be neutral. Always have to put on a fucking show to entertain men and keep them from attacking you."
]

In [15]:
preds = predict_sexism_generative_model(
    model_name = ModelsGenerativeEnum.MISTRAL,
    texts = texts,
    classification_prompt = "binary"
)
print(preds)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/1
	Time: 0.35 seconds
[0, 1, 1, 1, 1, 1]


In [13]:
preds = predict_sexism_generative_model(
    model_name = ModelsGenerativeEnum.MISTRAL,
    texts = texts,
    classification_prompt = "3labels"
)
print(preds)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/1
	Time: 0.41 seconds
[0, 2, 2, 2, 2, 2]


In [14]:
preds = predict_sexism_generative_model(
    model_name = ModelsGenerativeEnum.MISTRAL,
    texts = texts,
    classification_prompt = "4labels"
)
print(preds)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/1
	Time: 0.57 seconds
[0, 3, 1, 2, 3, 3]


In [ ]:
classification_prompt = "3labels"
dataset = "synthetic_phrases_sexism.csv"
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA_SEXISM, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.MISTRAL, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.GEMMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)

                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Batch 1/8
	Time: 0.56 seconds
Batch 2/8
	Time: 0.23 seconds
Batch 3/8
	Time: 0.23 seconds
Batch 4/8
	Time: 0.23 seconds
Batch 5/8
	Time: 0.23 seconds
Batch 6/8
	Time: 0.24 seconds
Batch 7/8
	Time: 0.23 seconds
Batch 8/8
	Time: 0.11 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_3labels_gemma-2-2b.csv


In [ ]:
classification_prompt = "binary"
dataset = "synthetic_phrases_sexism.csv"
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA_SEXISM, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.MISTRAL, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.GEMMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)

                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Batch 1/8
	Time: 0.25 seconds
Batch 2/8
	Time: 0.20 seconds
Batch 3/8
	Time: 0.19 seconds
Batch 4/8
	Time: 0.20 seconds
Batch 5/8
	Time: 0.19 seconds
Batch 6/8
	Time: 0.19 seconds
Batch 7/8
	Time: 0.19 seconds
Batch 8/8
	Time: 0.10 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_binary_llama-sexism-classifier-v1.csv
                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0
Batch 1/8
	Time: 0.10 seconds
Batch 2/8
	Time: 0.09 seconds
Batch 3/8
	Time: 0.09 seconds
Batch 4/8
	Time: 0.09 seconds
Batch 5/8
	Time: 0.09 seconds
Batch 6/8
	Time: 0.09 seconds
Batch 7/8
	Time: 0.09 seconds
Batch 8/8

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/8
	Time: 0.26 seconds
Batch 2/8
	Time: 0.21 seconds
Batch 3/8
	Time: 0.21 seconds
Batch 4/8
	Time: 0.21 seconds
Batch 5/8
	Time: 0.21 seconds
Batch 6/8
	Time: 0.21 seconds
Batch 7/8
	Time: 0.21 seconds
Batch 8/8
	Time: 0.10 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_binary_Mistral-7B-Instruct-v0.3.csv
                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/8
	Time: 0.24 seconds
Batch 2/8
	Time: 0.24 seconds
Batch 3/8
	Time: 0.24 seconds
Batch 4/8
	Time: 0.23 seconds
Batch 5/8
	Time: 0.23 seconds
Batch 6/8
	Time: 0.23 seconds
Batch 7/8
	Time: 0.24 seconds
Batch 8/8
	Time: 0.12 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_binary_gemma-2-2b.csv


In [ ]:
classification_prompt = "binary"
dataset = "edos_labelled_reduced.csv"
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA_SEXISM, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.MISTRAL, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.GEMMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)

                     rewire_id               text label_sexist  \
3933  sexism2022_english-17673        Fucking egg   not sexist   
4700  sexism2022_english-19879      My mom does 😊   not sexist   
1403   sexism2022_english-5827     We ban females       sexist   
3456   sexism2022_english-6787    No she's taken.   not sexist   
4832  sexism2022_english-16328  She is a cutie :3   not sexist   

                 label_category  \
3933                       none   
4700                       none   
1403  4. prejudiced discussions   
3456                       none   
4832                       none   

                                           label_vector split  
3933                                               none  test  
4700                                               none  test  
1403  4.2 supporting systemic discrimination against...  test  
3456                                               none  test  
4832                                               none  test  


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Batch 1/252
	Time: 0.24 seconds
Batch 2/252
	Time: 0.19 seconds
Batch 3/252
	Time: 0.19 seconds
Batch 4/252
	Time: 0.19 seconds
Batch 5/252
	Time: 0.19 seconds
Batch 6/252
	Time: 0.19 seconds
Batch 7/252
	Time: 0.19 seconds
Batch 8/252
	Time: 0.19 seconds
Batch 9/252
	Time: 0.19 seconds
Batch 10/252
	Time: 0.19 seconds
Batch 11/252
	Time: 0.19 seconds
Batch 12/252
	Time: 0.19 seconds
Batch 13/252
	Time: 0.19 seconds
Batch 14/252
	Time: 0.20 seconds
Batch 15/252
	Time: 0.20 seconds
Batch 16/252
	Time: 0.19 seconds
Batch 17/252
	Time: 0.19 seconds
Batch 18/252
	Time: 0.19 seconds
Batch 19/252
	Time: 0.19 seconds
Batch 20/252
	Time: 0.20 seconds
Batch 21/252
	Time: 0.19 seconds
Batch 22/252
	Time: 0.19 seconds
Batch 23/252
	Time: 0.19 seconds
Batch 24/252
	Time: 0.19 seconds
Batch 25/252
	Time: 0.19 seconds
Batch 26/252
	Time: 0.19 seconds
Batch 27/252
	Time: 0.19 seconds
Batch 28/252
	Time: 0.19 seconds
Batch 29/252
	Time: 0.19 seconds
Batch 30/252
	Time: 0.19 seconds
Batch 31/252
	Time:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/252
	Time: 0.26 seconds
Batch 2/252
	Time: 0.21 seconds
Batch 3/252
	Time: 0.21 seconds
Batch 4/252
	Time: 0.21 seconds
Batch 5/252
	Time: 0.21 seconds
Batch 6/252
	Time: 0.21 seconds
Batch 7/252
	Time: 0.21 seconds
Batch 8/252
	Time: 0.21 seconds
Batch 9/252
	Time: 0.21 seconds
Batch 10/252
	Time: 0.21 seconds
Batch 11/252
	Time: 0.21 seconds
Batch 12/252
	Time: 0.21 seconds
Batch 13/252
	Time: 0.21 seconds
Batch 14/252
	Time: 0.21 seconds
Batch 15/252
	Time: 0.21 seconds
Batch 16/252
	Time: 0.21 seconds
Batch 17/252
	Time: 0.21 seconds
Batch 18/252
	Time: 0.21 seconds
Batch 19/252
	Time: 0.21 seconds
Batch 20/252
	Time: 0.21 seconds
Batch 21/252
	Time: 0.21 seconds
Batch 22/252
	Time: 0.21 seconds
Batch 23/252
	Time: 0.21 seconds
Batch 24/252
	Time: 0.21 seconds
Batch 25/252
	Time: 0.21 seconds
Batch 26/252
	Time: 0.21 seconds
Batch 27/252
	Time: 0.21 seconds
Batch 28/252
	Time: 0.21 seconds
Batch 29/252
	Time: 0.21 seconds
Batch 30/252
	Time: 0.21 seconds
Batch 31/252
	Time:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/252
	Time: 0.28 seconds
Batch 2/252
	Time: 0.23 seconds
Batch 3/252
	Time: 0.23 seconds
Batch 4/252
	Time: 0.24 seconds
Batch 5/252
	Time: 0.23 seconds
Batch 6/252
	Time: 0.23 seconds
Batch 7/252
	Time: 0.23 seconds
Batch 8/252
	Time: 0.23 seconds
Batch 9/252
	Time: 0.23 seconds
Batch 10/252
	Time: 0.23 seconds
Batch 11/252
	Time: 0.23 seconds
Batch 12/252
	Time: 0.23 seconds
Batch 13/252
	Time: 0.24 seconds
Batch 14/252
	Time: 0.23 seconds
Batch 15/252
	Time: 0.23 seconds
Batch 16/252
	Time: 0.23 seconds
Batch 17/252
	Time: 0.23 seconds
Batch 18/252
	Time: 0.23 seconds
Batch 19/252
	Time: 0.23 seconds
Batch 20/252
	Time: 0.23 seconds
Batch 21/252
	Time: 0.23 seconds
Batch 22/252
	Time: 0.24 seconds
Batch 23/252
	Time: 0.23 seconds
Batch 24/252
	Time: 0.23 seconds
Batch 25/252
	Time: 0.23 seconds
Batch 26/252
	Time: 0.24 seconds
Batch 27/252
	Time: 0.23 seconds
Batch 28/252
	Time: 0.24 seconds
Batch 29/252
	Time: 0.24 seconds
Batch 30/252
	Time: 0.24 seconds
Batch 31/252
	Time:

In [ ]:
classification_prompt = "4labels"
dataset = "edos_labelled_4_sexism_grade_reduced.csv"
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA_SEXISM, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.MISTRAL, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.GEMMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)


                     rewire_id              text label_sexist  \
2141   sexism2022_english-5421              zits   not sexist   
4688   sexism2022_english-3897             lazy.   not sexist   
6232  sexism2022_english-11201        dis bitch.       sexist   
6464  sexism2022_english-16830   lock her ass up       sexist   
1330  sexism2022_english-13041  that is a wookie   not sexist   

                                label_category  \
2141                                      none   
4688                                      none   
6232                              3. animosity   
6464  1. threats, plans to harm and incitement   
1330                                      none   

                                           label_vector split  \
2141                                               none  test   
4688                                               none  test   
6232  3.1 casual use of gendered slurs, profanities,...  test   
6464           1.2 incitement and encouragement 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Batch 1/394
	Time: 0.40 seconds
Batch 2/394
	Time: 0.35 seconds
Batch 3/394
	Time: 0.35 seconds
Batch 4/394
	Time: 0.35 seconds
Batch 5/394
	Time: 0.35 seconds
Batch 6/394
	Time: 0.35 seconds
Batch 7/394
	Time: 0.35 seconds
Batch 8/394
	Time: 0.35 seconds
Batch 9/394
	Time: 0.35 seconds
Batch 10/394
	Time: 0.35 seconds
Batch 11/394
	Time: 0.35 seconds
Batch 12/394
	Time: 0.35 seconds
Batch 13/394
	Time: 0.35 seconds
Batch 14/394
	Time: 0.35 seconds
Batch 15/394
	Time: 0.36 seconds
Batch 16/394
	Time: 0.35 seconds
Batch 17/394
	Time: 0.35 seconds
Batch 18/394
	Time: 0.36 seconds
Batch 19/394
	Time: 0.36 seconds
Batch 20/394
	Time: 0.35 seconds
Batch 21/394
	Time: 0.35 seconds
Batch 22/394
	Time: 0.35 seconds
Batch 23/394
	Time: 0.35 seconds
Batch 24/394
	Time: 0.35 seconds
Batch 25/394
	Time: 0.36 seconds
Batch 26/394
	Time: 0.35 seconds
Batch 27/394
	Time: 0.36 seconds
Batch 28/394
	Time: 0.35 seconds
Batch 29/394
	Time: 0.35 seconds
Batch 30/394
	Time: 0.36 seconds
Batch 31/394
	Time:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/394
	Time: 0.38 seconds
Batch 2/394
	Time: 0.33 seconds
Batch 3/394
	Time: 0.33 seconds
Batch 4/394
	Time: 0.33 seconds
Batch 5/394
	Time: 0.33 seconds
Batch 6/394
	Time: 0.33 seconds
Batch 7/394
	Time: 0.33 seconds
Batch 8/394
	Time: 0.34 seconds
Batch 9/394
	Time: 0.34 seconds
Batch 10/394
	Time: 0.34 seconds
Batch 11/394
	Time: 0.34 seconds
Batch 12/394
	Time: 0.34 seconds
Batch 13/394
	Time: 0.34 seconds
Batch 14/394
	Time: 0.34 seconds
Batch 15/394
	Time: 0.34 seconds
Batch 16/394
	Time: 0.33 seconds
Batch 17/394
	Time: 0.33 seconds
Batch 18/394
	Time: 0.34 seconds
Batch 19/394
	Time: 0.33 seconds
Batch 20/394
	Time: 0.33 seconds
Batch 21/394
	Time: 0.33 seconds
Batch 22/394
	Time: 0.33 seconds
Batch 23/394
	Time: 0.33 seconds
Batch 24/394
	Time: 0.33 seconds
Batch 25/394
	Time: 0.33 seconds
Batch 26/394
	Time: 0.33 seconds
Batch 27/394
	Time: 0.33 seconds
Batch 28/394
	Time: 0.33 seconds
Batch 29/394
	Time: 0.33 seconds
Batch 30/394
	Time: 0.34 seconds
Batch 31/394
	Time:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/394
	Time: 0.26 seconds
Batch 2/394
	Time: 0.25 seconds
Batch 3/394
	Time: 0.25 seconds
Batch 4/394
	Time: 0.25 seconds
Batch 5/394
	Time: 0.25 seconds
Batch 6/394
	Time: 0.25 seconds
Batch 7/394
	Time: 0.25 seconds
Batch 8/394
	Time: 0.24 seconds
Batch 9/394
	Time: 0.24 seconds
Batch 10/394
	Time: 0.24 seconds
Batch 11/394
	Time: 0.24 seconds
Batch 12/394
	Time: 0.24 seconds
Batch 13/394
	Time: 0.24 seconds
Batch 14/394
	Time: 0.24 seconds
Batch 15/394
	Time: 0.24 seconds
Batch 16/394
	Time: 0.24 seconds
Batch 17/394
	Time: 0.24 seconds
Batch 18/394
	Time: 0.24 seconds
Batch 19/394
	Time: 0.24 seconds
Batch 20/394
	Time: 0.24 seconds
Batch 21/394
	Time: 0.24 seconds
Batch 22/394
	Time: 0.24 seconds
Batch 23/394
	Time: 0.24 seconds
Batch 24/394
	Time: 0.24 seconds
Batch 25/394
	Time: 0.24 seconds
Batch 26/394
	Time: 0.24 seconds
Batch 27/394
	Time: 0.24 seconds
Batch 28/394
	Time: 0.24 seconds
Batch 29/394
	Time: 0.24 seconds
Batch 30/394
	Time: 0.24 seconds
Batch 31/394
	Time:

In [ ]:
classification_prompt = "4labels"
dataset = "reddit_comments_gender_female_processed_phrase_annotated.csv"
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA_SEXISM, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.MISTRAL, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.GEMMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)

      attribute_in_window                 text               phrase  \
1873                 True               #NAME?               #NAME?   
574                  True        ur mom is art        ur mom is art   
2024                 True      my mom is nurse      my mom is nurse   
1175                 True   my wife is a nurse   my wife is a nurse   
1162                 True  my wife is a nurse.  my wife is a nurse.   

      label_sexist  bias_phrase  Unnamed: 5  
1873             2            2         NaN  
574              0            0         NaN  
2024             1            1         NaN  
1175             1            1         NaN  
1162             1            1         NaN  


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Batch 1/750
	Time: 0.38 seconds
Batch 2/750
	Time: 0.35 seconds
Batch 3/750
	Time: 0.35 seconds
Batch 4/750
	Time: 0.35 seconds
Batch 5/750
	Time: 0.35 seconds
Batch 6/750
	Time: 0.35 seconds
Batch 7/750
	Time: 0.35 seconds
Batch 8/750
	Time: 0.35 seconds
Batch 9/750
	Time: 0.35 seconds
Batch 10/750
	Time: 0.35 seconds
Batch 11/750
	Time: 0.35 seconds
Batch 12/750
	Time: 0.35 seconds
Batch 13/750
	Time: 0.35 seconds
Batch 14/750
	Time: 0.35 seconds
Batch 15/750
	Time: 0.35 seconds
Batch 16/750
	Time: 0.35 seconds
Batch 17/750
	Time: 0.35 seconds
Batch 18/750
	Time: 0.35 seconds
Batch 19/750
	Time: 0.35 seconds
Batch 20/750
	Time: 0.35 seconds
Batch 21/750
	Time: 0.35 seconds
Batch 22/750
	Time: 0.35 seconds
Batch 23/750
	Time: 0.35 seconds
Batch 24/750
	Time: 0.35 seconds
Batch 25/750
	Time: 0.35 seconds
Batch 26/750
	Time: 0.35 seconds
Batch 27/750
	Time: 0.35 seconds
Batch 28/750
	Time: 0.35 seconds
Batch 29/750
	Time: 0.35 seconds
Batch 30/750
	Time: 0.35 seconds
Batch 31/750
	Time:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/750
	Time: 0.36 seconds
Batch 2/750
	Time: 0.34 seconds
Batch 3/750
	Time: 0.34 seconds
Batch 4/750
	Time: 0.34 seconds
Batch 5/750
	Time: 0.34 seconds
Batch 6/750
	Time: 0.34 seconds
Batch 7/750
	Time: 0.34 seconds
Batch 8/750
	Time: 0.33 seconds
Batch 9/750
	Time: 0.33 seconds
Batch 10/750
	Time: 0.34 seconds
Batch 11/750
	Time: 0.33 seconds
Batch 12/750
	Time: 0.33 seconds
Batch 13/750
	Time: 0.34 seconds
Batch 14/750
	Time: 0.33 seconds
Batch 15/750
	Time: 0.33 seconds
Batch 16/750
	Time: 0.34 seconds
Batch 17/750
	Time: 0.33 seconds
Batch 18/750
	Time: 0.33 seconds
Batch 19/750
	Time: 0.33 seconds
Batch 20/750
	Time: 0.34 seconds
Batch 21/750
	Time: 0.34 seconds
Batch 22/750
	Time: 0.33 seconds
Batch 23/750
	Time: 0.33 seconds
Batch 24/750
	Time: 0.34 seconds
Batch 25/750
	Time: 0.33 seconds
Batch 26/750
	Time: 0.34 seconds
Batch 27/750
	Time: 0.34 seconds
Batch 28/750
	Time: 0.34 seconds
Batch 29/750
	Time: 0.33 seconds
Batch 30/750
	Time: 0.33 seconds
Batch 31/750
	Time:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/750
	Time: 0.25 seconds
Batch 2/750
	Time: 0.24 seconds
Batch 3/750
	Time: 0.24 seconds
Batch 4/750
	Time: 0.23 seconds
Batch 5/750
	Time: 0.24 seconds
Batch 6/750
	Time: 0.23 seconds
Batch 7/750
	Time: 0.24 seconds
Batch 8/750
	Time: 0.24 seconds
Batch 9/750
	Time: 0.24 seconds
Batch 10/750
	Time: 0.24 seconds
Batch 11/750
	Time: 0.24 seconds
Batch 12/750
	Time: 0.24 seconds
Batch 13/750
	Time: 0.24 seconds
Batch 14/750
	Time: 0.24 seconds
Batch 15/750
	Time: 0.24 seconds
Batch 16/750
	Time: 0.24 seconds
Batch 17/750
	Time: 0.24 seconds
Batch 18/750
	Time: 0.24 seconds
Batch 19/750
	Time: 0.24 seconds
Batch 20/750
	Time: 0.24 seconds
Batch 21/750
	Time: 0.24 seconds
Batch 22/750
	Time: 0.24 seconds
Batch 23/750
	Time: 0.24 seconds
Batch 24/750
	Time: 0.24 seconds
Batch 25/750
	Time: 0.24 seconds
Batch 26/750
	Time: 0.24 seconds
Batch 27/750
	Time: 0.24 seconds
Batch 28/750
	Time: 0.24 seconds
Batch 29/750
	Time: 0.24 seconds
Batch 30/750
	Time: 0.24 seconds
Batch 31/750
	Time:

In [ ]:
classification_prompt = "4labels"
dataset = "synthetic_phrases_sexism.csv"
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA_SEXISM, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.LLAMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.MISTRAL, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)
results = generative_classifier(model=ModelsGenerativeEnum.GEMMA, classification_prompt=classification_prompt, dataset=dataset, batch_size=4, limit=10000)

                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Batch 1/8
	Time: 0.38 seconds
Batch 2/8
	Time: 0.36 seconds
Batch 3/8
	Time: 0.35 seconds
Batch 4/8
	Time: 0.36 seconds
Batch 5/8
	Time: 0.35 seconds
Batch 6/8
	Time: 0.37 seconds
Batch 7/8
	Time: 0.37 seconds
Batch 8/8
	Time: 0.19 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_4labels_llama-sexism-classifier-v1.csv
                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0
Batch 1/8
	Time: 0.10 seconds
Batch 2/8
	Time: 0.10 seconds
Batch 3/8
	Time: 0.10 seconds
Batch 4/8
	Time: 0.10 seconds
Batch 5/8
	Time: 0.10 seconds
Batch 6/8
	Time: 0.10 seconds
Batch 7/8
	Time: 0.10 seconds
Batch 8/

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/8
	Time: 0.37 seconds
Batch 2/8
	Time: 0.33 seconds
Batch 3/8
	Time: 0.33 seconds
Batch 4/8
	Time: 0.33 seconds
Batch 5/8
	Time: 0.33 seconds
Batch 6/8
	Time: 0.34 seconds
Batch 7/8
	Time: 0.33 seconds
Batch 8/8
	Time: 0.17 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_4labels_Mistral-7B-Instruct-v0.3.csv
                                              text  manual_score
6                 A strong leader is always a man.           1.0
0            Women are not as good at math as men.           1.0
7         Women are not made for high-stress jobs.           1.0
8    It's unprofessional for men to cry in public.           1.0
2  Engineering jobs are too complicated for women.           1.0


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batch 1/8
	Time: 0.25 seconds
Batch 2/8
	Time: 0.24 seconds
Batch 3/8
	Time: 0.24 seconds
Batch 4/8
	Time: 0.24 seconds
Batch 5/8
	Time: 0.24 seconds
Batch 6/8
	Time: 0.24 seconds
Batch 7/8
	Time: 0.24 seconds
Batch 8/8
	Time: 0.12 seconds
Predicciones guardadas en /content/drive/My Drive/Facultad/Máster/TFM/Datos/synthetic_phrases_sexism/predictions_4labels_gemma-2-2b.csv


In [ ]:
from google.colab import runtime
runtime.unassign()          # disconnects and deletes the current runtime
